# Finance and Graphics Demo
## Calculate Bollinger Bands
## Graph with MatplotLib 
## Use a widget to make it interactive


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt 

import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [ ]:
try:
    import yfinance as yf
except: 
    !pip install yfinance
    import yfinance as yf 


## Let's download a stocks Dataset


In [ ]:
# Define the tickers for multiple stocks
tickers = ["AAPL", "GOOG", "AMZN", "META", "NVDA", "MSFT"]

# Download historical data for the past 5 years
stock_data = yf.download(tickers, start="2019-01-01", end="2025-02-19", group_by="ticker")

# Save the data to a CSV file (optional)
stock_data.to_csv("stock_data.csv")

# Display the first few rows
stock_data

In [ ]:
# Extract AAPL data
ap_prices = stock_data['AAPL'].copy()
ap_prices

# Bollinger Bands Explained

Bollinger Bands are a **technical analysis tool** used to measure market volatility and identify overbought or oversold conditions. They consist of three lines:

1. **Middle Band** – A simple moving average (SMA), typically a **20-day SMA**.
2. **Upper Band** – The SMA plus **2 standard deviations**.
3. **Lower Band** – The SMA minus **2 standard deviations**.

Since standard deviation measures volatility, the bands expand when the market is volatile and contract when the market is stable.

---

## **Formula for Bollinger Bands**
Given a time period \( N \) (typically 20 days):

- **Middle Band (SMA)**:  
  $$ \text{SMA} = \frac{1}{N} \sum_{i=1}^{N} P_i $$  
  where \( P_i \) is the price at day \( i \).

- **Upper Band**:  
  $$ \text{Upper Band} = \text{SMA} + (k \times \sigma) $$

- **Lower Band**:  
  $$ \text{Lower Band} = \text{SMA} - (k \times \sigma) $$
where:
-  $\sigma $ is the standard deviation of prices over $ N $ days.
- $k$ is typically **2**, meaning bands are **2 standard deviations** from the SMA.

---

## **How to Interpret Bollinger Bands**
1. **Price Near the Upper Band → Overbought**
   - If the price touches or moves above the upper band, the asset **may be overbought**, signaling a possible reversal downward.

2. **Price Near the Lower Band → Oversold**
   - If the price touches or moves below the lower band, the asset **may be oversold**, signaling a possible upward reversal.

3. **Bollinger Band Squeeze → Low Volatility**
   - When bands contract, it signals **low volatility** and often precedes a breakout in either direction.

4. **Bollinger Band Expansion → High Volatility**
   - When bands widen, volatility is increasing. This often happens after a strong move in price.

---


## Calculate the new Bollinger Bands columns
`Pandas` has a `.rolling` argument that we can pass in

In [ ]:
window = 20  

In [ ]:
ap_prices['SMA'] = ap_prices['Close'].rolling(window).mean()  # Simple Moving Average
ap_prices['StdDev'] = ap_prices['Close'].rolling(window).std()  # Standard Deviation
ap_prices['Upper'] = ap_prices['SMA'] + (2 * ap_prices['StdDev'])  # Upper Band
ap_prices['Lower'] = ap_prices['SMA'] - (2 * ap_prices['StdDev'])  # Lower Band

In [ ]:
ap_prices = ap_prices.dropna()
ap_prices

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ap_prices.index, ap_prices['Close'], label="Closing Price", color='blue')
plt.plot(ap_prices.index, ap_prices['SMA'], label="SMA", color='black', linestyle="dashed")
plt.plot(ap_prices.index, ap_prices['Upper'], label="Upper Band", color='red')
plt.plot(ap_prices.index, ap_prices['Lower'], label="Lower Band", color='green')
plt.fill_between(ap_prices.index, ap_prices['Upper'], ap_prices['Lower'], color='gray', alpha=0.2)
plt.title("Bollinger Bands (Apple 5 years)")
plt.legend()
plt.show()

In [ ]:
# Filter the data to only include the last year
last_year = ap_prices.loc[ap_prices.index >= ap_prices.index.max() - pd.DateOffset(years=1)]


In [ ]:
# Plot Bollinger Bands for the last year
plt.figure(figsize=(12, 6))
plt.plot(last_year.index, last_year['Close'], label="Closing Price", color='blue')
plt.plot(last_year.index, last_year['SMA'], label="SMA", color='black', linestyle="dashed")
plt.plot(last_year.index, last_year['Upper'], label="Upper Band", color='red')
plt.plot(last_year.index, last_year['Lower'], label="Lower Band", color='green')

# Fill the area between the upper and lower bands
plt.fill_between(last_year.index, last_year['Upper'].values, last_year['Lower'].values, color='gray', alpha=0.2)

plt.title("Bollinger Bands (Last Year)")
plt.legend()
plt.show()

## Identify overbought conditions where the closing price is above the upper Bollinger Band

In [ ]:
overbought = last_year[last_year['Close'] > last_year['Upper']]

overbought

### Plot Bollinger Bands with overbought conditions highlighted


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(last_year.index, last_year['Close'], label="Closing Price", color='blue')
plt.plot(last_year.index, last_year['SMA'], label="SMA", color='black', linestyle="dashed")
plt.plot(last_year.index, last_year['Upper'], label="Upper Band", color='red')
plt.plot(last_year.index, last_year['Lower'], label="Lower Band", color='green')

# Fill the Bollinger Bands range
plt.fill_between(last_year.index, last_year['Upper'].values, last_year['Lower'].values, color='gray', alpha=0.2)

# Highlight overbought points
plt.scatter(overbought.index, overbought['Close'], color='RED', label="Overbought", marker='o')

plt.title("Bollinger Bands - Overbought Periods (Last Year)")
plt.legend()
plt.show()

## Lets build some widgets!

In [ ]:
tickers

In [ ]:
# Create dropdown for stock selection
stock_picker = widgets.Dropdown(
    options=tickers,
    value="AAPL",
    description="Stock:"
)

# Create slider for selecting the time window in years
time_window = widgets.IntSlider(
    value=1,  # Default to 1 year
    min=1,
    max=5,
    step=1,
    description="Years:"
)

In [ ]:
# Display widgets
display(stock_picker, time_window)

In [ ]:
# Load stock data from CSV (Ensure the file is in the same directory)
#stock_data = pd.read_csv("stock_data.csv", header=[0, 1], index_col=0, parse_dates=True)

# Extract available tickers from the CSV columns
#tickers = [col[0] for col in stock_data.columns.levels[0]

In [ ]:
# Function to compute and plot Bollinger Bands
def plot_bollinger_bands(stock, years):
    # Extract only the selected stock's data
    df = stock_data[stock].copy()
    
    # Get the last date in the dataset and compute the start date
    end_date = df.index.max()
    start_date = end_date - pd.DateOffset(years=years)
    
    # Filter data based on the selected time window
    df = df.loc[start_date:end_date]

    # Compute Bollinger Bands
    window = 20  # Rolling window size
    df['SMA'] = df['Close'].rolling(window).mean()
    df['StdDev'] = df['Close'].rolling(window).std()
    df['Upper'] = df['SMA'] + (2 * df['StdDev'])
    df['Lower'] = df['SMA'] - (2 * df['StdDev'])

    # Drop NaN values
    df = df.dropna()

    # Plot Bollinger Bands
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Close'], label="Closing Price", color='blue')
    plt.plot(df.index, df['SMA'], label="SMA", color='black', linestyle="dashed")
    plt.plot(df.index, df['Upper'], label="Upper Band", color='red')
    plt.plot(df.index, df['Lower'], label="Lower Band", color='green')
    plt.fill_between(df.index, df['Upper'], df['Lower'], color='gray', alpha=0.2)

    plt.title(f"Bollinger Bands for {stock} ({years} Year(s))")
    plt.legend()
    plt.show()



In [ ]:
# Create an interactive widget with output
output = widgets.Output()

# Function to update the output widget
def update_plot(stock, years):
    with output:
        output.clear_output(wait=True)
        plot_bollinger_bands(stock, years)

# Create interactive widgets
interactive_plot = widgets.interactive(update_plot, stock=stock_picker, years=time_window)

# Display widgets and output
display(stock_picker, time_window, output)

# Call the function once to display initial plot
update_plot(stock_picker.value, time_window.value)